# ДЗ 5

In [1]:
import pandas as pd
import numpy as np

import torch
from rectools.dataset import Dataset

from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import SASRecModel, BERT4RecModel
from rectools.dataset import Dataset, Interactions

In [2]:
df_train = pd.read_csv('hse_train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4842338 entries, 0 to 4842337
Data columns (total 3 columns):
 #   Column     Dtype
---  ------     -----
 0   user_id    int64
 1   item_id    int64
 2   timestamp  int64
dtypes: int64(3)
memory usage: 110.8 MB


In [3]:
df_sub = pd.read_csv('sub0.csv')
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14039620 entries, 0 to 14039619
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   items    int64
dtypes: int64(2)
memory usage: 214.2 MB


In [4]:
df_train.describe()

,user_id,item_id,timestamp
count,4.842338e+06,4.842338e+06,4.842338e+06
mean,3.514203e+05,9.009362e+04,1.511887e+09
std,2.023899e+05,5.216975e+04,1.998496e+05
min,0.000000e+00,0.000000e+00,1.511539e+09
25%,1.762372e+05,4.469100e+04,1.511705e+09
50%,3.514285e+05,9.000700e+04,1.511876e+09
75%,5.265050e+05,1.352680e+05,1.512050e+09
max,7.019800e+05,1.806590e+05,1.512259e+09


In [5]:
df_train.head()

,user_id,item_id,timestamp
0,258671,74254,1511701649
1,258671,115615,1511841435
2,258671,176624,1512105022
3,240498,45484,1511605442
4,240498,39504,1511756830


In [6]:
df_sub.head()

,user_id,items
0,258671,1233
1,258671,119169
2,258671,131247
3,258671,105133
4,258671,180111


## Mose Popular

In [7]:
def baseline_most_popular(train_df, top_n=10):
    item_popularity = train_df['item_id'].value_counts().reset_index()
    item_popularity.columns = ['item_id', 'count']
  
    top_items = item_popularity.sort_values('count', ascending=False).head(top_n)['item_id'].values
    return list(top_items)

top_10_items = baseline_most_popular(df_train, top_n=10)
all_users = df_train['user_id'].unique()
recommendations = {user: top_10_items for user in all_users}


In [11]:
def prepare_submission(recommendations, sample_sub):
    sub_data = []
    for user, items in recommendations.items():
        sub_data.append({'user_id': user, 'items': items})
    submission = pd.DataFrame(sub_data)
    if sample_sub is not None:
        submission = submission[submission['user_id'].isin(sample_sub['user_id'])]
    
    return submission

In [ ]:

submission = prepare_submission(recommendations, df_sub)

submission.to_csv('baseline_popularity_submission.csv', index=False)
print("Baseline submission file saved!")

Baseline submission file saved!


In [23]:
submission.head()

,user_id,items
0,258671,100454
1,240498,100454
2,512761,100454
3,259030,100454
4,584368,100454


## User Similiriatty

In [14]:
from scipy.sparse import csr_matrix, lil_matrix

def baseline_user_similarity_sparse(train_df, top_n=10):
    user_ids = train_df['user_id'].unique()
    item_ids = train_df['item_id'].unique()
    
    user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
    item_to_idx = {item: idx for idx, item in enumerate(item_ids)}
    idx_to_item = {idx: item for item, idx in item_to_idx.items()}
    
    num_users = len(user_ids)
    num_items = len(item_ids)
    user_item_matrix = lil_matrix((num_users, num_items), dtype=np.int8)
    
    for _, row in train_df.iterrows():
        user_idx = user_to_idx[row['user_id']]
        item_idx = item_to_idx[row['item_id']]
        user_item_matrix[user_idx, item_idx] = 1  
    
    user_item_matrix_csr = user_item_matrix.tocsr()
    
    item_popularity = np.array(user_item_matrix_csr.sum(axis=0)).flatten()
    
    global_top_items = np.argsort(-item_popularity)[:top_n]
    global_top_items = [idx_to_item[idx] for idx in global_top_items]
    
    user_recommendations = {}
    for user in user_ids:
        user_idx = user_to_idx[user]
        
        _, items_bought = user_item_matrix_csr[user_idx].nonzero()
        
        if len(items_bought) == 0:
            user_recommendations[user] = global_top_items
            continue
        
        bought_popularity = item_popularity[items_bought]
        sorted_indices = np.argsort(-bought_popularity)
        top_personal_items = items_bought[sorted_indices[:top_n]]
        
        recommended_items = [idx_to_item[idx] for idx in top_personal_items]
        
        if len(recommended_items) < top_n:
            additional = [item for item in global_top_items 
                         if item not in recommended_items][:top_n - len(recommended_items)]
            recommended_items.extend(additional)
        
        user_recommendations[user] = recommended_items
    
    return user_recommendations

personal_recommendations = baseline_user_similarity_sparse(df_train, top_n=20)


In [18]:
personal_submission = prepare_submission(personal_recommendations, df_sub)
personal_submission.explode('items').reset_index(drop=True).to_csv('sparse_submission.csv', index=False)
print("Sparse matrix baseline submission file saved!")

Sparse matrix baseline submission file saved!


In [17]:
personal_submission.explode('items').reset_index(drop=True).head()

,user_id,items
0,258671,176624
1,258671,74254
2,258671,115615
3,258671,100454
4,258671,42316


In [ ]:
personal_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701981 entries, 0 to 701980
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  701981 non-null  int64 
 1   items    701981 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.7+ MB


## Sasrec

In [5]:
df_train["weight"] = 1.0

interactions = Interactions(
    df_train.rename(columns={
        "user_id": Columns.User,
        "item_id": Columns.Item,
        "timestamp": Columns.Datetime,
        "weight": Columns.Weight
    })
)

dataset = Dataset.construct(interactions.df)

In [61]:
model_rec_v2 = SASRecModel(
    session_max_len=20,
    loss="softmax",
    n_factors=64,
    batch_size=128,
    deterministic=True,
    epochs=3,
    verbose=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [62]:
model_rec_v2.fit(dataset)

/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/rectools/models/nn/item_net.py:134: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)
/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=('rectools.models.nn.item...net.CatFeaturesItemNet'), input_type=tuple])
  return self.__pydantic_serializer__.to_python(
/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `v

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [34]:
model_sas = torch.load("sas_rec_model.pt", weights_only=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/rectools/models/nn/item_net.py:134: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)


In [65]:
users = interactions.df["user_id"].unique()
recs = model_rec_v2.recommend(
    users=df_train["user_id"].unique(),  
    dataset=dataset,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/rectools/models/nn/transformers/data_preparator.py:361: UserWarning: 1 target users were considered cold because of missing known items
  warnings.warn(explanation)
/home/xande/anaconda3/envs/hse-11/lib/python3.11/site-packages/rectools/models/base.py:695: UserWarning: 
                Model `<class 'rectools.models.nn.transformers.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)


In [66]:
personal_submission = prepare_submission(recs, df_sub)
personal_submission.to_csv('sasrec_submission_submission.csv', index=False)
print("SasRec file saved!")

SasRec file saved!


## IAls

In [78]:
import pandas as pd
import numpy as np
import implicit
from scipy.sparse import csr_matrix
from collections import defaultdict
from tqdm import tqdm

def prepare_data(train_df):
    user_ids = train_df['user_id'].unique()
    item_ids = train_df['item_id'].unique()
    
    user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
    item_to_idx = {item: idx for idx, item in enumerate(item_ids)}
    
    rows = train_df['user_id'].map(user_to_idx)
    cols = train_df['item_id'].map(item_to_idx)
    values = np.ones(len(train_df)) 
    
    user_item_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    return user_item_matrix, user_to_idx, item_to_idx, user_ids, item_ids

def train_als(user_item_matrix, factors=64, iterations=15, regularization=0.01):
    model = implicit.als.AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=regularization,
        random_state=42
    )
    model.fit(user_item_matrix)
    return model

def generate_recommendations(model, user_item_matrix, user_ids, item_ids, n_recommendations=1):
    user_to_recommendations = defaultdict(list)
    item_id_list = np.array(item_ids)
    
    for user_id in tqdm(user_ids): 
        ids, scores = model.recommend(user_id, user_item_matrix[user_id], N=n_recommendations)
        user_to_recommendations[user_id] = item_id_list[ids[0]]

    return user_to_recommendations

def prepare_submission(recommendations, submission_sample):
    submission = pd.DataFrame({
        'user_id': list(recommendations.keys()),
        'items': [' '.join(map(str, items)) for items in recommendations.values()]
    })
    submission = submission.sort_values('user_id').reset_index(drop=True)
    return submission

In [74]:
user_item_matrix, user_to_idx, item_to_idx, user_ids, item_ids = prepare_data(df_train)

model = train_als(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

In [79]:
recommendations = generate_recommendations(model, user_item_matrix, user_ids, item_ids, 1)

100%|██████████| 701981/701981 [10:18<00:00, 1135.76it/s]


In [27]:
def prepare_submission(recommendations, sample_sub):
    sub_data = []
    for user, items in recommendations.items():
        sub_data.append({'user_id': user, 'items': items})
    submission = pd.DataFrame(sub_data)
    if sample_sub is not None:
        submission = submission[submission['user_id'].isin(sample_sub['user_id'])]
    
    return submission

In [99]:
submission = prepare_submission(recommendations, df_sub)

submission.to_csv('als_baseline.csv', index=False)

In [86]:
submission.head()

,user_id,items
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [89]:
recommendations[258671]

40491

## ImplicitKNN

In [18]:
from rectools.models import ImplicitItemKNNWrapperModel
from implicit.nearest_neighbours import TFIDFRecommender

model = ImplicitItemKNNWrapperModel(TFIDFRecommender(K=10))
model.fit(dataset)

In [19]:
recos = model.recommend(
    users=df_train["user_id"].unique(),
    dataset=dataset,
    k=1,
    filter_viewed=True,
)

In [32]:
recos.head()

,user_id,item_id,score,rank
0,258671,88182,0.151825,1
1,240498,98991,0.376998,1
2,512761,43983,0.173695,1
3,259030,72204,0.127852,1
4,584368,87942,0.183311,1


In [48]:
def prepare_submission(recommendations, sample_sub):
    submission = recommendations[['user_id', 'item_id']].rename(columns={'item_id': 'items'})
    if sample_sub is not None:
        submission = submission[submission['user_id'].isin(sample_sub['user_id'])]
    
    return submission

In [49]:
personal_submission = prepare_submission(recos, df_sub)

In [50]:
personal_submission.head()

,user_id,items
0,258671,88182
1,240498,98991
2,512761,43983
3,259030,72204
4,584368,87942


In [51]:
personal_submission.to_csv('knn_submission_submission.csv', index=False)
print("KNN file saved!")

KNN file saved!
